# Projeto 1 - Ciência dos Dados

Nome: Larissa Jordana de Paula Soares

Nome: Luis Antônio Bordignon de Oliveira

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [39]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import emoji
import unidecode
import unicodedata
import re

In [40]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\LariLari\Projeto1_Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [41]:
filename = 'sony.xlsx'

In [42]:
train = pd.read_excel(filename)
train.head(6)

,Treinamento,Classificação
0,se o martinho da vila pedir 01 dólar por cada ...,I
1,@jeanino_ @s9_ricardo @fuck_off_matt então a s...,I
2,2022 vai me falir dmsssss pqp caralho https://...,I
3,"@luarzinmsm ata, a sony é uma vagabunda mesmo",I
4,@rafael_fenixx @raphaelvieirac4 @drakesincero1...,I
5,@formigonny acho que vai ser só sony viu...,I


In [43]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Clasificação
0,único jogo novo anunciado pela sony que eu que...,R
1,@highlevelbeu sony no seu nivel mais belo né k...,I
2,a sony vai lançar uma sequência de jogos mt fo...,I
3,@th0rdthy gloria a sony,I
4,"@paladinodoxbox sony liberou a versão do ps5, ...",I


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

RESPOSTA: Foram consideradas relevantes críticas à empresa sony (com contexto detectável), como a insatisfação do público por causa do pequeno tempo de duração de um filme produzido pela empresa. Também, em menor quantidade, elogios certeiros foram levados em consideração, apontando aonde a empresa acertou e deve continuar assim.


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [44]:
# Arquivo "sony"
train.head()

,Treinamento,Classificação
0,se o martinho da vila pedir 01 dólar por cada ...,I
1,@jeanino_ @s9_ricardo @fuck_off_matt então a s...,I
2,2022 vai me falir dmsssss pqp caralho https://...,I
3,"@luarzinmsm ata, a sony é uma vagabunda mesmo",I
4,@rafael_fenixx @raphaelvieirac4 @drakesincero1...,I


In [45]:
#célula de limpeza
import functools
import operator
def cleanup(text):#Função de limpeza muito simples que troca alguns sinais básicos por espaços
    #import string
    punctuation = '[!-.:?;"”“*)(/|[\]]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = emoji.get_emoji_regexp().split(text_subbed)
    separa = [texto.split() for texto in text_subbed]
    text_subbed = functools.reduce(operator.concat, separa)
    return " ".join(text_subbed)

def lower(text):#Fução que coloca todos texto em minúsculo 
    return text.lower()

# get_emoji_regxp pesquisar............
# 

def remove_imagem_arroba(text):
    lista = ""
    for i in text.split():
        if i[:4]!= 'http' and i[0]!='@':
            lista+= i + " "
    return lista.strip()

# função que remove acentuação
def remove_acentos(text):
    text = unicodedata.normalize('NFD', text)
    return re.sub(r'[\u0300-\u036f]', '', text).casefold()


In [46]:
def limpeza (text):
    text = remove_acentos(text)
    text= lower(text)
    text = remove_imagem_arroba(text)
    text = cleanup(text)
    return text

In [47]:
num= 0
for i in train.loc[:,'Treinamento']:
    train.loc[num, 'Treinamento']= limpeza(i)
    num+= 1
train

,Treinamento,Classificação
0,se o martinho da vila pedir 01 dolar por cada ...,I
1,entao a sony n revela os numeros de tlou pq fr...,I
2,2022 vai me falir dmsssss pqp caralho,I
3,ata a sony e uma vagabunda mesmo,I
4,ao decorrer das decadas sony god of war 1° fra...,I
...,...,...
295,q na verdade ja tem ne pq em 2017 passava na s...,I
296,relevancia me diga qual foi o ultimo jogo inov...,R
297,thor gordo isso mesmo sony sempre fiel a mitol...,I
298,estranho e saber que a r conseguiu ser a unica...,I


In [48]:
#Separando as mensagens Relevantes e Irrelevantes da nossa base de treinamento
Train_Relevante= train.loc[train['Classificação']=='R', 'Treinamento']
Train_Irrelevante= train.loc[train['Classificação']=='I', 'Treinamento']
Train_Total= train.loc[:,'Treinamento']
train.shape[0]

300

In [49]:
#Implementando a limpeza
l_relevante=[]
l_irrelevante=[]
l_total=[]

for i in Train_Relevante:
    l_relevante+= " ".join(i).split()

for i in Train_Irrelevante:
    l_irrelevante+= " ".join(i).split()

for i in Train_Total:
    l_total+= " ".join(i).split()

In [50]:
serie_train_R = pd.Series(l_relevante)
serie_train_I = pd.Series(l_irrelevante)
serie_train_T = pd.Series(l_total)

In [51]:
len(serie_train_T)

28713

In [52]:
#Frequencia absoluta
tab_train_R_abs = serie_train_R.value_counts()
tab_train_I_abs = serie_train_I.value_counts()
tab_train_T_abs = serie_train_T.value_counts()

In [53]:
tab_train_R_relativa = serie_train_R.value_counts(True)
tab_train_I_relativa = serie_train_I.value_counts(True)
tab_train_T_relativa = serie_train_T.value_counts(True)

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [54]:
#Calculo de P(R) e P(I)
probR = sum(tab_train_R_abs) / sum(tab_train_T_abs)

probI = sum(tab_train_I_abs) / sum(tab_train_T_abs)

In [55]:
#Fazendo a limpeza
l_test= []
for i in test.Teste:
    l_test+=(limpeza(i).split())

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

https://www.youtube.com/watch?v=F_gYj6fcSl4 **Ensina como remover os acentos**
https://gist.github.com/luizomf/54b58615cd674db44153470c369a8284 **git do Youtube**